# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [1]:
import pandas as pd

df = pd.read_csv('homepage_actions.csv')


In [9]:
# Display the first few rows of the dataset
print(df.head(20))

# Check the dimensions of the dataset (number of rows and columns)
print(df.shape)

# Check the data types of each column
print(df.dtypes)

# Check for missing values
print(df.isnull().sum())

# Check summary statistics of numerical columns
print(df.describe())


                     timestamp      id       group action
0   2016-09-24 17:42:27.839496  804196  experiment   view
1   2016-09-24 19:19:03.542569  434745  experiment   view
2   2016-09-24 19:36:00.944135  507599  experiment   view
3   2016-09-24 19:59:02.646620  671993     control   view
4   2016-09-24 20:26:14.466886  536734  experiment   view
5   2016-09-24 20:32:25.712659  681598  experiment   view
6   2016-09-24 20:39:03.248853  522116  experiment   view
7   2016-09-24 20:57:20.336757  349125  experiment   view
8   2016-09-24 20:58:01.948663  349125  experiment  click
9   2016-09-24 21:00:12.278374  560027     control   view
10  2016-09-24 21:00:41.351487  265695  experiment   view
11  2016-09-24 21:05:15.348935  601714  experiment   view
12  2016-09-24 21:06:27.553057  601714  experiment  click
13  2016-09-24 21:09:09.046127  416862  experiment   view
14  2016-09-24 21:29:19.766467  487634  experiment   view
15  2016-09-24 21:30:02.739756  487634  experiment  click
16  2016-09-24

In [4]:
#Investigate the 'id' column: Analyze the 'id' column to understand the relationship between viewers and clickers.


# Check the number of viewers and clickers
viewers = df['id'].nunique()
clickers = df[df['action'] == 'click']['id'].nunique()
print("Number of viewers:", viewers)
print("Number of clickers:", clickers)

# Check for any anomalies where someone clicked without viewing (if applicable)
anomalies = df[df['action'] == 'click']['id'].isin(df[df['action'] == 'view']['id']).sum()
print("Number of anomalies (click without view):", anomalies)


Number of viewers: 6328
Number of clickers: 1860
Number of anomalies (click without view): 1860


In [5]:
#Identify overlap between the control and experiment groups: Determine if there is any overlap between the control and experiment groups, and decide how to account for it in the experimental design.


# Checking for overlap between control and experiment groups
control_group = df[df['group'] == 'control']['id'].unique()
experiment_group = df[df['group'] == 'experiment']['id'].unique()
overlap = set(control_group).intersection(experiment_group)
print("Number of overlapping users:", len(overlap))


Number of overlapping users: 0


## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [10]:
import pandas as pd
import numpy as np
from statsmodels.stats.proportion import proportions_ztest

# Load the dataset
df = pd.read_csv('homepage_actions.csv')

# Subset the data for the control and experiment groups
control_group = df[df['group'] == 'control']
experiment_group = df[df['group'] == 'experiment']

# Count the number of conversions (clicks) and total observations (views) for each group
control_conversions = control_group[control_group['action'] == 'click'].shape[0]
control_total = control_group[control_group['action'] == 'view'].shape[0]

experiment_conversions = experiment_group[experiment_group['action'] == 'click'].shape[0]
experiment_total = experiment_group[experiment_group['action'] == 'view'].shape[0]

# Perform the two-sample proportion z-test
count = np.array([control_conversions, experiment_conversions])
nobs = np.array([control_total, experiment_total])
z_score, p_value = proportions_ztest(count, nobs)

# Define the significance level (alpha)
alpha = 0.05

# Interpret the results
if p_value < alpha:
    print("The experimental homepage was more effective than the control group (statistically significant).")
else:
    print("There is no significant difference between the experimental homepage and the control group.")

print("p-value:", p_value)
print("z-score:", z_score)




The experimental homepage was more effective than the control group (statistically significant).
p-value: 0.008830075576595804
z-score: -2.618563885349469


## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [11]:
# Calculating the click-through rate of the control group
control_click_through_rate = control_conversions / control_total

# Calculating the expected number of clicks for the experiment group
expected_experiment_clicks = control_click_through_rate * experiment_total

print("Expected number of clicks for the experiment group:", expected_experiment_clicks)


Expected number of clicks for the experiment group: 838.0168067226891


### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [12]:
# Calculating the variance for the binomial distribution
variance = experiment_total * control_click_through_rate * (1 - control_click_through_rate)

# Calculating the standard deviation
standard_deviation = np.sqrt(variance)

# Calculating the number of standard deviations from the estimate
z_score = (experiment_conversions - expected_experiment_clicks) / standard_deviation

print("Number of standard deviations from the estimate:", z_score)


Number of standard deviations from the estimate: 3.6625360854823588


### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [15]:
import numpy as np
from scipy.stats import norm


# Calculate the p-value
p_value = 1 - norm.cdf(z_score)

print("p-value based on the normal distribution:", p_value)


p-value based on the normal distribution: 0.00012486528006949715


### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment:

The result obtained from the analysis using the expected number of clicks, standard deviations, and p-value calculation based on the normal distribution is consistent with the previous statistical test. The p-value obtained from the binomial analysis (p = 0.00012486528006949715) is very close to the p-value obtained from the t-test (p = 0.008830075576595804). Both p-values indicate statistical significance, suggesting that the experimental homepage was more effective than the control group.

The number of standard deviations from the estimate (z-score) is also consistent, with a value of 3.6625360854823588. This indicates that the actual number of clicks in the experiment group is 3.66 standard deviations away from the expected number of clicks based on the control group's click-through rate.

Therefore, the analysis using the binary variable formulation and the binomial approach verifies the results of the previous statistical test, providing further evidence that the experimental homepage was more effective in terms of click-through rate compared to the control group.

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.